# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
L. Eisert  ->  L. Eisert  |  ['L. Eisert']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 57 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/1 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2306.01056


extracting tarball to tmp_2306.01056...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2306.01056/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'appendix/appendix' from 'tmp_2306.01056/appendix/appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'conclusion/conclusion' from 'tmp_2306.01056/conclusion/conclusion.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/

Found 68 bibliographic references in tmp_2306.01056/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.01056-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.01056) | **ERGO-ML: Towards a robust machine learning model for inferring the  fraction of accreted stars in galaxies from integral-field spectroscopic maps**  |
|| E. Angeloudi, et al. -- incl., <mark>A. Pillepich</mark>, <mark>L. Eisert</mark> |
|*Appeared on*| *2023-06-05*|
|*Comments*| *23 pages, 15 figures. Accepted for publication in MNRAS*|
|**Abstract**| Quantifying the contribution of mergers to the stellar mass of galaxies is key for constraining the mechanisms of galaxy assembly across cosmic time. However, the mapping between observable galaxy properties and merger histories is not trivial: cosmological galaxy simulations are the only tools we have for calibration. We study the robustness of a simulation-based inference of the ex-situ stellar mass fraction of nearby galaxies to different observables -- integrated and spatially-resolved -- and to different galaxy formation models -- IllustrisTNG and EAGLE -- with Machine Learning. We find that at fixed simulation, the fraction of accreted stars can be inferred with very high accuracy, with an error $\sim5$ per cent (10 per cent) from 2D integral-field spectroscopic maps (integrated quantities) throughout the considered stellar mass range. A bias (> 5 per cent) and an increase in scatter by a factor of 2 are introduced when testing with a different simulation, revealing a lack of generalization to distinct galaxy-formation models. Interestingly, upon using only stellar mass and kinematics maps in the central galactic regions for training, we find that this bias is removed and the ex-situ stellar mass fraction can be recovered in both simulations with < 15 per cent scatter, independently of the training set's origin. This opens up the door to a potential robust inference of the accretion histories of galaxies from existing Integral Field Unit surveys, such as MaNGA, covering a similar field of view (FOV) and containing spatially-resolved spectra for tens of thousands of nearby galaxies. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2306.01056.md
    + _build/html/tmp_2306.01056/./x_figures/Relative_Errors/Channels_ALL_on_TNG.png
    + _build/html/tmp_2306.01056/./x_figures/Relative_Errors/Channels_ALL_on_EAGLE.png
    + _build/html/tmp_2306.01056/./x_figures/Balance/tng_hist.png
    + _build/html/tmp_2306.01056/./x_figures/Balance/eagle_hist.png
    + _build/html/tmp_2306.01056/./x_figures/Balance/tng_balanced_hist.png
    + _build/html/tmp_2306.01056/./x_figures/Balance/eagle_balanced_hist.png
    + _build/html/tmp_2306.01056/./x_figures/Catalog_predictions/TNG_on_TNG_model_All.png
    + _build/html/tmp_2306.01056/./x_figures/Groundtruth_vs_Predictions/TNG_on_TNG_model_All.png
    + _build/html/tmp_2306.01056/./x_figures/Catalog_predictions/EAGLE_on_EAGLE_model_All.png
    + _build/html/tmp_2306.01056/./x_figures/Groundtruth_vs_Predictions/EAGLE_on_EAGLE_model_All.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\rowname}[1]$
$\newcommand{\columnname}[1]$
$\newcommand{\ap}[1]{{\color{magenta} #1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# ERGO-ML: Towards a robust machine learning model for inferring the fraction of accreted stars in galaxies from integral-field spectroscopic maps

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2306.01056-b31b1b.svg)](https://arxiv.org/abs/2306.01056)<mark>Appeared on: 2023-06-05</mark> -  _23 pages, 15 figures. Accepted for publication in MNRAS_

</div>
<div id="authors">

E. Angeloudi, et al. -- incl., <mark>A. Pillepich</mark>, <mark>L. Eisert</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ Quantifying the contribution of mergers to the stellar mass of galaxies is key for constraining the mechanisms of galaxy assembly across cosmic time. However, the mapping between observable galaxy properties and merger histories is not trivial: cosmological galaxy simulations are the only  tools we have for calibration. We study the robustness of a simulation-based inference of the ex-situ stellar mass fraction of nearby galaxies to different observables -- integrated and spatially-resolved -- and to different galaxy formation models -- IllustrisTNG and EAGLE -- with Machine Learning. We find that at fixed simulation, the fraction of accreted stars can be inferred with very high accuracy, with an error $\sim5$ per cent (10 per cent) from 2D integral-field spectroscopic maps (integrated quantities) throughout the considered stellar mass range. A bias (> 5 per cent) and an increase in scatter by a factor of 2 are introduced when testing with a different simulation, revealing a lack of generalization to distinct galaxy-formation models. Interestingly, upon using only stellar mass and kinematics maps in the central galactic regions for training, we find that this bias is removed and the ex-situ stellar mass fraction can be recovered in both simulations with <15 per cent scatter, independently of the training set's origin. This opens up the door to a potential robust inference of the accretion histories of galaxies from existing Integral Field Unit surveys, such as MaNGA, covering a similar field of view (FOV) and containing spatially-resolved spectra for tens of thousands of nearby galaxies. $\$

</div>

<div id="div_fig1">

<img src="tmp_2306.01056/./x_figures/Relative_Errors/Channels_ALL_on_TNG.png" alt="Fig10.1" width="50%"/><img src="tmp_2306.01056/./x_figures/Relative_Errors/Channels_ALL_on_EAGLE.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** The prediction error (prediction - ground truth) versus the ex-situ stellar mass fraction ground-truth for all galaxies in the TNG100 (a) and EAGLE test set (b) for three different combinations of the input channels. For each panel the median of the over- or under-prediction is illustrated as a solid line and the shaded regions include the 68 per cent of the data points. The medians of the other channels combinations are also displayed in every panel for visual reference. **Top:** The prediction errors of models trained on three different input combinations when applied on the TNG100 test set. **Bottom:** The prediction errors of models trained on three different input combinations when applied on the EAGLE-L100 test set. In both sub-figures, we illustrate on the first column the results when testing and training in a single simulation setup and the second column corresponds to the cross-testing scenario across simulations. Each row corresponds to a distinct configuration of the input channels in the 2D maps. Spatially-resolved maps of stellar mass and kinematics prove to be the most robust predictors across simulations. (*fig:results_relative_error_channels*)

</div>
<div id="div_fig2">

<img src="tmp_2306.01056/./x_figures/Balance/tng_hist.png" alt="Fig1.1" width="25%"/><img src="tmp_2306.01056/./x_figures/Balance/eagle_hist.png" alt="Fig1.2" width="25%"/><img src="tmp_2306.01056/./x_figures/Balance/tng_balanced_hist.png" alt="Fig1.3" width="25%"/><img src="tmp_2306.01056/./x_figures/Balance/eagle_balanced_hist.png" alt="Fig1.4" width="25%"/>

**Figure 1. -** The distribution of the ex-situ stellar mass fraction values for the selected samples from the TNG100 and EAGLE-L100 simulations. We select all galaxies with stellar mass $> {10}^{10} {M}_{\odot }$ from $z = 0$(TNG100 and EAGLE-L100) and galaxies of the same stellar mass range with an ex-situ stellar mass fraction $f_{ex} > 0.2 $ from $z = 0.1$(TNG100 and EAGLE-L100) and $z = 0.2$(EAGLE-L100), in an initial attempt to balance our dataset samples. **Top:** Histogram of ex-situ stellar mass fractions of the original TNG100 (on the left) and EAGLE-L100 galaxy samples (on the right). This sample distribution is used for the integrated values inference approach. **Bottom:** Histogram of ex-situ mass fractions of the two simulation samples after balancing by using multiple projections per galaxy. This sample is used for the 2D spatially-resolved maps inference approach. (*fig:balancing_ds*)

</div>
<div id="div_fig3">

<img src="tmp_2306.01056/./x_figures/Catalog_predictions/TNG_on_TNG_model_All.png" alt="Fig2.1" width="25%"/><img src="tmp_2306.01056/./x_figures/Groundtruth_vs_Predictions/TNG_on_TNG_model_All.png" alt="Fig2.2" width="25%"/><img src="tmp_2306.01056/./x_figures/Catalog_predictions/EAGLE_on_EAGLE_model_All.png" alt="Fig2.3" width="25%"/><img src="tmp_2306.01056/./x_figures/Groundtruth_vs_Predictions/EAGLE_on_EAGLE_model_All.png" alt="Fig2.4" width="25%"/>

**Figure 2. -** Evaluation of models trained on a fixed simulation scenario with the integrated inputs (a, c) and the 2D spatial maps (b, d) for the TNG100 (upper row) and EAGLE-L100 (bottom row) test sets. In each panel, the ensemble predictions of the models versus the ground truth of the ex-situ stellar mass fraction is illustrated for 128 galaxies randomly selected from the two test sets. The error bars correspond to the standard deviation of the ensemble posterior that is predicted for each galaxy. The black dashed line on the prediction panels marks the 1:1 relation and serves as a guide to the eye, as the prediction points should be gathered as close to that line as possible. On the bottom error panels, the blue dashed line indicates the median of the error in the prediction, i.e. the difference between the model's estimate and the ground truth. The shaded region contains the 68 per cent of the data. (*fig:tng_eagle_test_set_results*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2306.01056"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

213  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
